In [1]:
from src import lcu

lcu = lcu.LCU()

In [2]:
srf_read, srf_str = lcu.asmToHex("NOP")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lcu.asmToHex("EXIT")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lcu.asmToHex("SADD R1, ZERO, LAST")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lcu.asmToHex("SADD R1, SRF(3), LAST")
print(str(srf_read) + ", " + str(srf_str))
#srf_read, srf_str = lcu.asmToHex("SADD R1, SRF(3), ONE") # Error
srf_read, srf_str = lcu.asmToHex("SADDI R1, ONE, 7")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lcu.asmToHex("SSUB SRF(4), SRF(4), SRF(4)")
print(str(srf_read) + ", " + str(srf_str))
#srf_read, srf_str = lcu.asmToHex("SSUB SRF(4), SRF(4), SRF(3)") # Error
srf_read, srf_str = lcu.asmToHex("JUMP ONE, 7")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lcu.asmToHex("BGEPD ONE, ZERO, 5")
print(str(srf_read) + ", " + str(srf_str))
for i in range(len(lcu.imem.IMEM)):
    print(lcu.imem.get_word_in_hex(i))

-1, -1
-1, -1
-1, -1
3, -1
-1, -1
4, 4
-1, -1
-1, -1
0x0
0x1c00
0xb8340
0xb0340
0xfc347
0x90400
0xfda07
0xdd605
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0


In [3]:
# Get instruction dataframe
from src import vwr2a
kernel_path='./kernels/test_asm/'
cgra = vwr2a.CGRA()
cgra.compileAsm(kernel_path)

AttributeError: module 'src.lsu' has no attribute 'asmToHex'

In [ ]:
from src import lsu

lsu = lsu.LSU()

In [ ]:
srf_read, srf_str = lsu.asmToHex("SADD R0, ONE, ONE/LD.VWR VWR_A")
print(str(srf_read) + ", " + str(srf_str))
srf_read, srf_str = lsu.asmToHex("SADD R0, SRF(5), ONE/SH.IL.UP")
print(str(srf_read) + ", " + str(srf_str))
#srf_read, srf_str = lsu.asmToHex("SADD SRF(4), SRF(5), ONE/SH.IL.UP") # Error
srf_read, srf_str = lsu.asmToHex("SADD SRF(5), SRF(5), ONE/LD.VWR SRF")
print(str(srf_read) + ", " + str(srf_str))
for i in range(len(lsu.imem.IMEM)):
    print(lsu.imem.get_word_in_hex(i))

-1, -1
5, -1
5, 5
0x45538
0xc4538
0x5c530
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80
0x4c80


In [1]:
# Get instruction dataframe
from src import vwr2a
kernel_path='./kernels/test_asm/'
cgra = vwr2a.CGRA()
cgra.compileAsm(kernel_path)